In [1]:
# 한국어를 지원하는 폰트로 변경
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)

print("완료!")

완료!


In [2]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

print(tf.__version__)

2.6.0


In [3]:
import tarfile

# 압축 풀기
tar_path = "./data/korean-english-park.train.tar.gz"
extract_path = "./data/korean-english-park/"

with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall(path=extract_path)

print("압축 해제 완료!")

압축 해제 완료!


In [4]:
import os

print(os.listdir(extract_path))

['korean-english-park.train.en', 'korean-english-park.train.ko']


In [5]:
# 파일 경로 설정
kor_file = "./data/korean-english-park/korean-english-park.train.ko"
eng_file = "./data/korean-english-park/korean-english-park.train.en"

# 변환된 파일 저장 경로
kor_output = "./data/korean.txt"
eng_output = "./data/english.txt"

# 한국어, 영어 문장을 각각 저장
with open(kor_file, "r", encoding="utf-8") as f_ko, open(eng_file, "r", encoding="utf-8") as f_en:
    korean_sentences = f_ko.readlines()
    english_sentences = f_en.readlines()

# 새로운 파일로 저장
with open(kor_output, "w", encoding="utf-8") as f_ko, open(eng_output, "w", encoding="utf-8") as f_en:
    f_ko.writelines(korean_sentences)
    f_en.writelines(english_sentences)

print("파일 변환 완료! 'korean.txt'와 'english.txt'가 생성되었습니다.")

파일 변환 완료! 'korean.txt'와 'english.txt'가 생성되었습니다.


In [6]:
import re
import MeCab

def clean_text(sentence, lang="ko"):
    """특수 문자 및 불필요한 공백 제거"""
    if lang == "ko":
        sentence = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣ\s]", "", sentence)  # 한글과 공백만 남기기
    else:  # 영어
        sentence = sentence.lower().strip()  # 소문자로 변환 및 공백 제거
        sentence = re.sub(r"[^a-zA-Z?.!,¿]", " ", sentence)  # 특수 문자 제거
    sentence = re.sub(r"\s+", " ", sentence).strip()  # 불필요한 공백 제거
    return sentence

# Step2 : 데이터 전처리

### 정제하기 & 토큰화하기
- 데이터는 \t 기호를 기준으로 영어와 스페인어가 병렬 쌍
- \t 기호를 매개변수로 split() 함수를 호출

In [7]:
import re
import MeCab

# MeCab 토크나이저 초기화
mecab = MeCab.Tagger("-Owakati")

def clean_text(sentence, lang="ko"):
    """특수 문자 및 불필요한 공백 제거"""
    if lang == "ko":
        sentence = re.sub(r"[^가-힣ㄱ-ㅎㅏ-ㅣ\s]", "", sentence)  # 한글과 공백만 남기기
    else:  # 영어
        sentence = sentence.lower().strip()  # 소문자로 변환 및 공백 제거
        sentence = re.sub(r"[^a-zA-Z?.!,¿]", " ", sentence)  # 특수 문자 제거
    sentence = re.sub(r"\s+", " ", sentence).strip()  # 불필요한 공백 제거
    return sentence

def tokenize_text(sentence, lang="ko"):
    """토큰화 수행"""
    if lang == "ko":
        return mecab.parse(sentence).strip()  # MeCab 토큰화
    else:  # 영어
        return "<start> " + sentence + " <end>"  # 시작과 끝 토큰 추가

In [8]:
# 파일 경로 설정
kor_path = "./data/korean.txt"
eng_path = "./data/english.txt"

# 중복 제거를 위한 set 활용
unique_pairs = set()

with open(kor_path, "r", encoding="utf-8") as f_kor, open(eng_path, "r", encoding="utf-8") as f_eng:
    for kor, eng in zip(f_kor, f_eng):
        kor_clean = clean_text(kor.strip(), lang="ko")
        eng_clean = clean_text(eng.strip(), lang="en")

        kor_tokenized = tokenize_text(kor_clean, lang="ko")
        eng_tokenized = tokenize_text(eng_clean, lang="en")

        # 병렬 데이터 쌍을 (한국어, 영어) 형태로 저장
        unique_pairs.add((kor_tokenized, eng_tokenized))

# 정제된 데이터 저장 (길이 필터링 포함)
cleaned_corpus = []
eng_corpus = []
kor_corpus = []

for kor, eng in unique_pairs:
    if len(kor.split()) <= 40 and len(eng.split()) <= 40:  # 길이 제한 적용
        cleaned_corpus.append((kor, eng))
        kor_corpus.append(kor)
        eng_corpus.append(eng)

print("✅ 데이터 정제 및 토큰화 완료!")
print(f"🔹 중복 제거 후 데이터 개수: {len(cleaned_corpus)}")
print("🔹 한국어 샘플:", kor_corpus[:3])
print("🔹 영어 샘플:", eng_corpus[:3])

✅ 데이터 정제 및 토큰화 완료!
🔹 중복 제거 후 데이터 개수: 66658
🔹 한국어 샘플: ['쿠바 서부 지역 의 토양 은 세계 에서 담배 를 기르 는 최적 의 장소 이 다', '남북한 은 년 역사 적 인 정상 회담 으로 남북 관계 가 해빙 되 기 전 수 십 년 동안 서로 를 비방 했 다', '매케인 은 우리 는 새롭 고 간단 한 세금 시스템 을 만들 어 미국인 들 에게 기회 를 주 려고 한다고 덧붙였 다']
🔹 영어 샘플: ['<start> the soil in the western part of the country is considered the best in the world for growing tobacco. <end>', '<start> before their historic summit in thawed relations, the two koreas had vilified each other for decades. <end>', '<start> we are going to create a new and simpler tax system and give the american people a choice. <end>']


In [9]:
import tensorflow as tf

# 토큰화 함수
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')   # 특수문자 필터링 X, num_words 추후 설정
    tokenizer.fit_on_texts(corpus)  # 단어 사전 구축
 
    tensor = tokenizer.texts_to_sequences(corpus)  # 문장을 정수 시퀀스로 변환
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  # 패딩 추가

    return tensor, tokenizer

# 한국어 문장 (입력) 토큰화
enc_tensor, enc_tokenizer = tokenize(kor_corpus)

# 영어 문장 (출력) 토큰화
dec_tensor, dec_tokenizer = tokenize(eng_corpus)

# 데이터 확인
print("한국어 vocab size:", len(enc_tokenizer.word_index) + 1)  # 패딩 포함
print("영어 vocab size:", len(dec_tokenizer.word_index) + 1)
print("Training samples:", len(enc_tensor))


한국어 vocab size: 38544
영어 vocab size: 61539
Training samples: 66658


# Step3 : 모델 설계

In [10]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
    
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)

        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        attn = tf.nn.softmax(score, axis=1)

        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)

        return context_vec, attn

In [11]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units,
                                       return_sequences=True)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.gru(out)
        
        return out

In [12]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units,
                                       return_sequences=True,
                                       return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, h_dec, enc_out):
        context_vec, attn = self.attention(enc_out, h_dec)

        out = self.embedding(x)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)
        
        out, h_dec = self.gru(out)
        out = tf.reshape(out, (-1, out.shape[2]))
        out = self.fc(out)

        return out, h_dec, attn

In [17]:
# 코드를 실행하세요.

BATCH_SIZE     = 32
SRC_VOCAB_SIZE = len(enc_tokenizer.index_word) + 1
TGT_VOCAB_SIZE = len(dec_tokenizer.index_word) + 1

units         = 1024
embedding_dim = 512

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                     sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

Encoder Output: (32, 30, 1024)
Decoder Output: (32, 61539)
Decoder Hidden State: (32, 1024)
Attention: (32, 30, 1)


### Optimizer & Loss

In [18]:
# Optimizer와 Loss 정의
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))  # 0을 패딩으로 간주하고 마스크 생성
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask  # 마스크를 곱하여 패딩 부분을 제외한 손실 계산
    
    return tf.reduce_mean(loss)

### train step 구현

In [19]:
@tf.function
def train_step(src, tgt, encoder, decoder, optimizer, dec_tokenizer):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape:
        enc_out = encoder(src)  # Encoder의 출력
        h_dec = enc_out[:, -1]  # 마지막 hidden state
        
        dec_src = tf.expand_dims([dec_tokenizer.word_index['<start>']] * bsz, 1)  # <start> 토큰을 decoder에 전달

        for t in range(1, tgt.shape[1]):  # t: 시퀀스의 각 timestep
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)  # Decoder 실행

            loss += loss_function(tgt[:, t], pred)  # 손실 계산
            dec_src = tf.expand_dims(tgt[:, t], 1)  # 다음 time step으로 이동

    batch_loss = loss / int(tgt.shape[1])  # 시퀀스 길이에 따른 평균 손실 계산

    variables = encoder.trainable_variables + decoder.trainable_variables  # 학습 가능한 변수들
    gradients = tape.gradient(loss, variables)  # 그래디언트 계산
    optimizer.apply_gradients(zip(gradients, variables))  # 그래디언트 적용
    
    return batch_loss

### 훈련 시작하기

In [20]:
import random
from tqdm import tqdm 

EPOCHS = 10

# 훈련 루프
for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_tensor.shape[0], BATCH_SIZE))  # 데이터 인덱스 리스트
    random.shuffle(idx_list)  # 데이터 셔플
    t = tqdm(idx_list)  # 진행 표시줄

    for batch, idx in enumerate(t):
        batch_loss = train_step(enc_tensor[idx:idx+BATCH_SIZE],
                                dec_tensor[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                dec_tokenizer)  # 훈련 단계 실행
    
        total_loss += batch_loss
        
        # tqdm 업데이트
        t.set_description(f'Epoch {epoch + 1}')  # 에폭 표시
        t.set_postfix(loss=f'{total_loss.numpy() / (batch + 1):.4f}')  # 현재 에폭에 대한 손실 표시

  0%|          | 0/2084 [00:51<?, ?it/s]


ResourceExhaustedError:    OOM when allocating tensor with shape[1024,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node split_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[decoder_1/gru_3/PartitionedCall_24]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_step_144992]

Function call stack:
train_step -> train_step -> train_step


In [ ]:
# 훈련 후 예문 번역
    print("=== Translation for Test Sentences ===")
    test_sentences = ["오바마는 대통령이다.", "시민들은 도시 속에 산다.", "커피는 필요 없다.", "일곱 명의 사망자가 발생했다."]
    for sentence in test_sentences:
        translate(sentence, encoder, decoder)

In [ ]:
# 번역 및 Attention 시각화
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((max_dec_len, max_enc_len))  # max_dec_len과 max_enc_len 정의 필요
    
    sentence = preprocess_sentence(sentence)
    inputs = enc_tokenizer.texts_to_sequences([sentence.split()])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                           maxlen=max_enc_len,
                                                           padding='post')

    result = ''

    enc_out = encoder(inputs)

    dec_hidden = enc_out[:, -1]
    dec_input = tf.expand_dims([dec_tokenizer.word_index['<start>']], 0)

    for t in range(max_dec_len):  # max_dec_len 정의 필요
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention[t] = attention_weights.numpy()

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0]).numpy()

        result += dec_tokenizer.index_word[predicted_id] + ' '

        if dec_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention


def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    
    attention = attention[:len(result.split()), :len(sentence.split())]
    plot_attention(attention, sentence.split(), result.split(' '))

# 번역 예시 실행
translate("오바마는 대통령이다.", encoder, decoder)
translate("시민들은 도시 속에 산다.", encoder, decoder)
translate("커피는 필요 없다.", encoder, decoder)
translate("일곱 명의 사망자가 발생했다.", encoder, decoder)